In [1]:
from openai import OpenAI
import os
import json
from tqdm import tqdm
import time
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import instructor # I'm not sure if this will actually be needed...
from pydantic import BaseModel, Field
from typing import List, Dict

In [2]:
load_dotenv('../.env')

openai_api_key = os.getenv('OPENAI_API_KEY')

# Test OpenAI API...

client = OpenAI(
  api_key=openai_api_key,  # this is also the default, it can be omitted
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Can you state the name of a real species of bird? Only reply with the species name.",
        }
    ],
    model="gpt-4o-mini",
)

result = chat_completion.choices[0].message.content
print(result)

Pica pica


# Plan of action

- Create R scripts that extract a) main files, b) supplementary files, c) metadata. Separately.
- I suspect I would want to download and output the structure of these files... I had a passing thought that just looking at the name of the file could be sufficient (e.g. save time/space associated with downloading a file), but given the possible heterogeneity of data in GEO, I think it would be beneficial to download the actual data and visualise the output
- After seeing the output, I could ask the LLM "is this a counts matrix..."
- But I'm also aware that some (all probably...) files come as zipped, so I would need to unzip them.

In [ ]:
import subprocess

# Define the path to the R script
r_script_path = "./RScript_GetGEOData.r"  # Ensure the path is correct and script is executable

# Define the GEO accession you want to use
geo_accession = "GSE3494"  # Replace with the actual GEO accession number

# Construct the command to run the R script with the GEO accession as an argument
command = ["Rscript", r_script_path, "-g", geo_accession]

# Call the R script using subprocess
try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print("R script output:")
    print(result.stdout)  # This will print the standard output from the R script
except subprocess.CalledProcessError as e:
    print("Error running R script:")
    print(e.stderr)  # This will print any error messages from the R script